In [0]:
%sql
create connection if not exists earthquake_conn
type HTTP
options
(
  host = 'https://earthquake.usgs.gov',
  port = 443,
  base_path = '/earthquakes/feed/v1.0/',
  bearer_token = 'na'
)

In [0]:
%py
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
conn = w.connections.get("earthquake_conn")
base_url = f"{conn.options['host']}{conn.options['base_path']}"


In [0]:
dbutils.widgets.text('catalog_name','dev','dev')
catalog_name = dbutils.widgets.get('catalog_name')
print(catalog_name)

In [0]:
spark.sql(
    f"use catalog {catalog_name}"
)
spark.sql(
    "use schema bronze"
)
spark.sql(
    "create volume if not exists earthquake_vol"
)




In [0]:
import requests
import json
import datetime
url = f'{base_url}/summary/all_day.geojson'
response = requests.get(url)
if response.status_code != 200:
  raise Exception(f"Error getting data from {url}")
current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
data = response.json() 
dbutils.fs.put(f'/Volumes/{catalog_name}/bronze/earthquake_vol/earthquake_data_{current_time}.json',json.dumps(data),overwrite=True)